In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/Applied CV Project/fruits-360_dataset/fruits-360

/content/drive/.shortcut-targets-by-id/1i0IgUBpr8uyXxhLQgE5RaQK7Qf6agpOh/Applied CV Project/fruits-360_dataset/fruits-360


In [ ]:
ls ./Training

'Apple Braeburn'/        Clementine/          Mangostan/             Pineapple/
'Apple Crimson Snow'/    Cocos/               Maracuja/             'Pineapple Mini'/
'Apple Golden 1'/        Corn/               'Melon Piel de Sapo'/  'Pitahaya Red'/
'Apple Golden 2'/       'Corn Husk'/          Mulberry/              Plum/
'Apple Golden 3'/       'Cucumber Ripe'/      Nectarine/            'Plum 2'/
'Apple Granny Smith'/   'Cucumber Ripe 2'/   'Nectarine Flat'/      'Plum 3'/
'Apple Pink Lady'/       Dates/              'Nut Forest'/           Pomegranate/
'Apple Red 1'/           Eggplant/           'Nut Pecan'/           'Pomelo Sweetie'/
'Apple Red 2'/           Fig/                'Onion Red'/           'Potato Red'/
'Apple Red 3'/          'Ginger Root'/       'Onion Red Peeled'/    'Potato Red Washed'/
'Apple Red Delicious'/   Granadilla/         'Onion White'/         'Potato Sweet'/
'Apple Red Yellow 1'/   'Grape Blue'/         Orange/               'Potato White'/
'Apple Red Y

In [ ]:
ls ./Test

'Apple Braeburn'/        Clementine/          Mangostan/             Pineapple/
'Apple Crimson Snow'/    Cocos/               Maracuja/             'Pineapple Mini'/
'Apple Golden 1'/        Corn/               'Melon Piel de Sapo'/  'Pitahaya Red'/
'Apple Golden 2'/       'Corn Husk'/          Mulberry/              Plum/
'Apple Golden 3'/       'Cucumber Ripe'/      Nectarine/            'Plum 2'/
'Apple Granny Smith'/   'Cucumber Ripe 2'/   'Nectarine Flat'/      'Plum 3'/
'Apple Pink Lady'/       Dates/              'Nut Forest'/           Pomegranate/
'Apple Red 1'/           Eggplant/           'Nut Pecan'/           'Pomelo Sweetie'/
'Apple Red 2'/           Fig/                'Onion Red'/           'Potato Red'/
'Apple Red 3'/          'Ginger Root'/       'Onion Red Peeled'/    'Potato Red Washed'/
'Apple Red Delicious'/   Granadilla/         'Onion White'/         'Potato Sweet'/
'Apple Red Yellow 1'/   'Grape Blue'/         Orange/               'Potato White'/
'Apple Red Y

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-j1n4gazc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-j1n4gazc
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [ ]:
import json
import os
from PIL import Image

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

In [ ]:
class image_title_dataset(Dataset):
    def __init__(self, list_image_path, list_txt):
        self.image_path = list_image_path
        self.title = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        try:
            image = preprocess(Image.open(self.image_path[idx]))
        except FileNotFoundError:
            print(f"File not found: {self.image_path[idx]}. Skipping...")
            return None, None

        title = self.title[idx]
        return image, title

In [ ]:
def createDataLoader(json_path):
    with open(json_path, "r") as file:
        json_data = json.load(file)

    input_data = []
    for item in json_data:
        input_data.append(item)

    list_image_path = []
    list_txt = []
    for item in input_data:
        img_path = item['file_path']
        if os.path.exists(img_path):
            caption = item['labels'][0]
            list_image_path.append(img_path)
            list_txt.append(caption)
        else:
            print(f"File not found: {img_path}. Skipping...")

    dataset = image_title_dataset(list_image_path, list_txt)
    dataloader = DataLoader(dataset, batch_size=100, shuffle=True, drop_last=True)
    return dataloader

In [ ]:
train_path = './Training_data.json'
test_path = './Test_data.json'

In [ ]:
cd Test

/content/drive/.shortcut-targets-by-id/1i0IgUBpr8uyXxhLQgE5RaQK7Qf6agpOh/Applied CV Project/fruits-360_dataset/fruits-360/Test


In [ ]:
test_dataloader = createDataLoader(test_path)

In [ ]:
cd ../Training

/content/drive/.shortcut-targets-by-id/1i0IgUBpr8uyXxhLQgE5RaQK7Qf6agpOh/Applied CV Project/fruits-360_dataset/fruits-360/Training


In [ ]:
train_dataloader = createDataLoader(train_path)

File not found: ./Peach Flat (1)/r_236_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_122_100.jpg. Skipping...
File not found: ./Peach Flat (1)/310_100.jpg. Skipping...
File not found: ./Peach Flat (1)/213_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_313_100.jpg. Skipping...
File not found: ./Peach Flat (1)/18_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_149_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_159_100.jpg. Skipping...
File not found: ./Peach Flat (1)/188_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_303_100.jpg. Skipping...
File not found: ./Peach Flat (1)/27_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_182_100.jpg. Skipping...
File not found: ./Peach Flat (1)/11_100.jpg. Skipping...
File not found: ./Peach Flat (1)/r_192_100.jpg. Skipping...
File not found: ./Peach Flat (1)/285_100.jpg. Skipping...
File not found: ./Peach Flat (1)/295_100.jpg. Skipping...
File not found: ./Peach Flat (1)/278_100.jpg. Skipping...
F

In [ ]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()


if device == "cpu":
  model.float()

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

In [ ]:
def train_model(num_epochs):
  for epoch in range(num_epochs):
      pbar = tqdm(train_dataloader, total=len(train_dataloader))
      for batch in pbar:
          optimizer.zero_grad()

          images,texts = batch

          images= images.to(device)
          texts = texts.to(device)

          logits_per_image, logits_per_text = model(images, texts)

          ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
          total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

          total_loss.backward()
          if device == "cpu":
              optimizer.step()
          else :
              convert_models_to_fp32(model)
              optimizer.step()
              clip.model.convert_weights(model)

          pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}")

In [ ]:
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            images, texts = batch
            images = images.to(device)
            texts = texts.to(device)

            logits_per_image, logits_per_text = model(images, texts)
            _, predicted = torch.max(logits_per_image + logits_per_text, 1)
            total += texts.size(0)
            correct += (predicted == torch.arange(len(images), dtype=torch.long, device=device)).sum().item()

    accuracy = correct / total
    return accuracy

In [ ]:
train_model(1)
print(evaluate_model(model))

Epoch 0/1, Loss: 0.9707: 100%|██████████| 673/673 [12:34:19<00:00, 67.25s/it]

File not found: ./Cactus fruit/212_100.jpg. Skipping...
File not found: ./Huckleberry/r_40_100.jpg. Skipping...
File not found: ./Nectarine/r_322_100.jpg. Skipping...
File not found: ./Kaki/r_171_100.jpg. Skipping...
File not found: ./Apricot/r_69_100.jpg. Skipping...
File not found: ./Redcurrant/r_157_100.jpg. Skipping...
File not found: ./Kaki/r_192_100.jpg. Skipping...
File not found: ./Plum 2/62_100.jpg. Skipping...
File not found: ./Orange/r_96_100.jpg. Skipping...
File not found: ./Fig/r_192_100.jpg. Skipping...
File not found: ./Cantaloupe 1/82_100.jpg. Skipping...
File not found: ./Pear Forelle/r2_169_100.jpg. Skipping...
File not found: ./Mandarine/r_245_100.jpg. Skipping...
File not found: ./Corn/r2_164_100.jpg. Skipping...
File not found: ./Tangelo/r_287_100.jpg. Skipping...
File not found: ./Tomato 1/r2_9_100.jpg. Skipping...
File not found: ./Salak/280_100.jpg. Skipping...
File not found: ./Walnut/13_100.jpg. Skipping...
File not found: ./Cherry Wax Black/192_100.jpg. Skip

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>